# Exploratory Data Analysis

Here, I will just check if I can get some usable data from the GDELT 2.0 Event
Database via Google BigQuery.

So far, I didn't decide for which problem to solve and what kind of model to
train, because I want to find a suitable dataset first.

Here's what I'm looking for data:
- with time stamps that's updated regularly, so I can train an initial model
and then schedule it to run periodically and monitor it
- that's sufficiently large to train a model on
- that has some interesting features and a suitable target variable

During a brief search, I found the GDELT 2.0 Event Database, which is a public
and free database that contains event data from all over the world.
It seems to fulfill these requirements and is available via BigQuery.

Here, I will check if I can get some data from it and if it's suitable for my
needs.

## Environment

To use this project's uv environment, make sure you installed it according to
the instructions in the README.md file.

Then, connect to the `.venv` kernel.
Check the path to the kernel to make sure it's the right one.
It should be `.venv/bin/python`.

Run the next cell to check if you use the correct kernel.
It should output this:

# FIXME: Once I decided for an actual name for the repo, adapt the path!
```
<path_to_wherever_you_cloned_the_repo_to>/mlopsproject2/.venv/bin/python
```

In [2]:
!which python

/Users/fakrueg/projects/courses/datatalks/mlops-zoomcamp/mlopsproject2/.venv/bin/python


In [ ]:
# Dependencies
import os
import pandas as pd
from google.cloud import bigquery
import pandas_gbq
from dotenv import load_dotenv
from pathlib import Path
from sklearn.model_selection import train_test_split

# Load environment variables
load_dotenv()

True

In [84]:
# define paths
PATH_REPO = Path(".").resolve().parent
PATH_DATA = PATH_REPO / "data" / "raw"

In [ ]:
# BigQuery Client Setup
def setup_bigquery_client():
    """
    Set up BigQuery client using credentials file
    """
    # Check if credentials file exists
    cred_path = "../bigquery-credentials.json"
    if not Path(cred_path).exists():
        raise FileNotFoundError(f"Credentials file not found: {cred_path}")
    
    # Set environment variable for this session
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = cred_path
    
    # Get project ID from environment
    project_id = os.getenv('GOOGLE_CLOUD_PROJECT')
    if not project_id:
        raise ValueError("GOOGLE_CLOUD_PROJECT not set in .env file")
    
    # Initialize client
    client = bigquery.Client(project=project_id)
    return client

# Initialize BigQuery client
try:
    client = setup_bigquery_client()
    print(f"BigQuery client initialized successfully!")
    print(f"Project: {client.project}")
    print(f"Using credentials from: ./bigquery-credentials.json")
except Exception as e:
    print(f"Error setting up BigQuery client: {e}")
    client = None

BigQuery client initialized successfully!
Project: mlops-zoomcamp-1337420697
Using credentials from: ./bigquery-credentials.json


# FIXME: Explain why I selected these exact features.

In [70]:
def safe_gdelt_query(start_date, end_date, limit=100, dry_run=True):
    """
    Safely query GDELT data with automatic cost estimation
    
    Args:
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format  
        limit (int): Maximum number of rows to return
        dry_run (bool): If True, only estimate query cost
    """

    if client is None:
        raise ValueError("BigQuery client not initialized")
    
    # Convert dates to GDELT format (YYYYMMDD) as integers
    start_gdelt = int(start_date.replace('-', ''))
    end_gdelt = int(end_date.replace('-', ''))

    query = f"""
    SELECT 
        SQLDATE,                -- event date
        MonthYear,              -- month and year
        EventCode,
        EventBaseCode,
        EventRootCode,
        QuadClass,
        GoldsteinScale,
        Actor1Code,
        Actor1Name,
        Actor1CountryCode,
        Actor1Type1Code,
        Actor1Type2Code,
        Actor1Type3Code,
        Actor2Code,
        Actor2Name,
        Actor2CountryCode,
        Actor2Type1Code,
        Actor2Type2Code,
        Actor2Type3Code,
        ActionGeo_CountryCode,
        ActionGeo_ADM1Code,
        ActionGeo_Lat,
        ActionGeo_Long,
        ActionGeo_FeatureID,
        NumArticles             -- target variable
    FROM `gdelt-bq.gdeltv2.events`
    WHERE SQLDATE >= {start_gdelt}  -- start date
      AND SQLDATE <= {end_gdelt}    -- end date
    ORDER BY RAND()                 -- order rows randomly to get random sample
    LIMIT {limit}                   -- limit the number of rows to return
    """

    # Always do a dry run first for cost estimation
    job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
    dry_job = client.query(query, job_config=job_config)

    bytes_processed = dry_job.total_bytes_processed
    estimated_cost = (bytes_processed / 1e12) * 5  # $5 per TB

    print(
        f"Query will process: {bytes_processed:,} bytes "
        f"({bytes_processed/1e6:.2f} MB) or rather "
        f"({bytes_processed/1e9:.2f} GB)."
    )
    print(f"Estimated cost: ${estimated_cost:.6f}")

    if dry_run:
        print("Dry run complete - no data retrieved")
        return None

    # Execute the actual query
    print("Executing query...")
    df = pd.read_gbq(query, project_id=client.project, dialect='standard')

    print(f"Query completed! Retrieved {len(df)} rows")
    return df

Get 10k random rows from GDELT events table from year 2024.

I decided to go for a sample size 10k rows, because that should be an acceptable
balance between speed of model training and showing it enough data.
If I go for an 80:20 train:test split, I will end up with 8k rows for training
and 2k rows for testing.
There are just 24 features and one target variable.
So basically the ration rows to features is 10000:24, which is 416.67.
I intend to use tree based algorithms such as XGBoost, CatBoost and LightGBM.
They are rather data efficient, and at this ratio, maybe it's even already
enough for acceptable performance.

Honestly, I could go for **much** more than that though, but then models would
train much longer, too.
This is some sort of a subset for speed of development.
At the same time, I could have also gone for much less than that, but then it
would definitely become a true subset, and whatever I train would likely be
underperforming.
So I decide to go with this as a compromise and check how well it performs.
If it does good enough, I won't need to go for a larger subset.
If it doesn't perform well, I can at least select hyperparameters and then go
for a larger subset.
Then again, this is not a machine learning engineering course, but a machine
learning *operations*, so I don't need to get the best possible model in the
first place.
A good model is sufficient.

In [ ]:
# Start with just a dry run to check costs
test_df = safe_gdelt_query(
    '2024-01-01',
    '2024-12-31',
    limit=14000,
    dry_run=True
)

Query will process: 101,663,870,564 bytes (101663.87 MB) or rather (101.66 GB).
Estimated cost: $0.508319
Dry run complete - no data retrieved


In [ ]:
# Looks fine enough, so go for it
# loads a pandas df into object data_gdelt

# actually I just added this False as another layer of safety, so it doesn't
# automatically run stuff
# BigQuery can generate some costs, but at this rate it won't, because we're
# still well below the free quota of 1TB per month
if False:
    data_gdelt = safe_gdelt_query(
        '2024-01-01',
        '2024-12-31',
        limit=10000,
        dry_run=False
    )

Query will process: 101,664,169,301 bytes (101664.17 MB) or rather (101.66 GB).
Estimated cost: $0.508321
Executing query...


/var/folders/mk/sqk63g391x1cj_llvqsv9xfmkb6y8r/T/ipykernel_61614/564667602.py:73: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df = pd.read_gbq(query, project_id=client.project, dialect='standard')


Query completed! Retrieved 10000 rows


In [ ]:
# just have a look at if downloading worked
data_gdelt.head()

,SQLDATE,MonthYear,EventCode,EventBaseCode,EventRootCode,QuadClass,GoldsteinScale,Actor1Code,Actor1Name,Actor1CountryCode,...,Actor2CountryCode,Actor2Type1Code,Actor2Type2Code,Actor2Type3Code,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,NumArticles
0,20241127,202411,111,111,11,3,-2.0,None,None,None,...,RUS,MIL,None,None,RS,RS48,55.7522,37.6156,-2960561,10
1,20240520,202405,030,030,03,1,4.0,CVL,VOTER,None,...,None,None,None,None,US,USCA,34.1819,-118.3600,273472,2
2,20240417,202404,061,061,06,2,6.4,GBR,BRITAIN,GBR,...,None,None,None,None,NZ,NZ,-42.0000,174.0000,NZ,5
3,20240916,202409,190,190,19,4,-10.0,HTI,HAITI,HTI,...,None,CVL,None,None,HA,HA11,18.5392,-72.3350,-70311,5
4,20240524,202405,046,046,04,1,7.0,None,None,None,...,MRT,None,None,None,MR,MR06,18.1194,-16.0406,-1402901,2


In [90]:
# write data to parquet, so I can re-use it later without querying again
data_gdelt.to_parquet(
    PATH_DATA / "gdelt_events_2024_subset_10k_full.parquet",
    index=False
)

In [91]:
# load data from parquet again
data_gdelt = pd.read_parquet(
    PATH_DATA / "gdelt_events_2024_subset_10k_full.parquet"
)

## Split data into train and test

Split the data first to prevent data leakage.
Make a truly unseen hold out test set, which will not be used for training or
validation at all.
It will only be used to evaluate one single final model in the very end.

I will use a 80:20 split for training and testing.
This will leave me with 8k rows for training and 2k rows for testing.
For development, I will use 5-fold cross validation.

In [ ]:
# split data into train and test
train_df, test_df = train_test_split(
    data_gdelt,
    test_size=0.2,
    random_state=42
)

# check the data
train_df.head()

,SQLDATE,MonthYear,EventCode,EventBaseCode,EventRootCode,QuadClass,GoldsteinScale,Actor1Code,Actor1Name,Actor1CountryCode,...,Actor2CountryCode,Actor2Type1Code,Actor2Type2Code,Actor2Type3Code,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,NumArticles
9254,20240328,202403,050,050,05,1,3.5,BUS,COMPANIES,None,...,None,None,None,None,SO,SO,6.0000,48.00000,SO,8
1561,20240515,202405,040,040,04,1,1.0,LEG,REPRESENTATIVES,None,...,FRA,GOV,None,None,FR,FR00,48.8667,2.33333,-1456928,3
1670,20240405,202404,036,036,03,1,4.0,ESP,SPAIN,ESP,...,None,GOV,None,None,SP,SP,40.0000,-4.00000,SP,10
6087,20240209,202402,114,114,11,3,-2.0,USA,NORTH CAROLINA,USA,...,None,LEG,None,None,US,USNC,35.6411,-79.84310,NC,4
6669,20240815,202408,042,042,04,1,1.9,TZA,TANZANIA,TZA,...,KEN,None,None,None,TZ,TZ,-6.0000,35.00000,TZ,1


In [93]:
# save both train and test data
train_df.to_parquet(
    PATH_DATA / "gdelt_events_2024_subset_10k_train.parquet",
    index=False
)

test_df.to_parquet(
    PATH_DATA / "gdelt_events_2024_subset_10k_test.parquet",
    index=False
)

Now I can work on the train split without risking leaking any information.

Check for missing values first. 
If there is any column that has a lot of missing values, I will drop it.
Columns with just a low percentages may be imputed if there is a meaningful
way.

In [ ]:
# check for missing values
train_df.isnull().mean()

SQLDATE                  0.000000
MonthYear                0.000000
EventCode                0.000000
EventBaseCode            0.000000
EventRootCode            0.000000
QuadClass                0.000000
GoldsteinScale           0.000000
Actor1Code               0.102375
Actor1Name               0.102375
Actor1CountryCode        0.460875
Actor1Type1Code          0.551750
Actor1Type2Code          0.975000
Actor1Type3Code          0.999250
Actor2Code               0.304250
Actor2Name               0.304250
Actor2CountryCode        0.566375
Actor2Type1Code          0.671500
Actor2Type2Code          0.983375
Actor2Type3Code          0.999125
ActionGeo_CountryCode    0.030625
ActionGeo_ADM1Code       0.030625
ActionGeo_Lat            0.032375
ActionGeo_Long           0.031750
ActionGeo_FeatureID      0.030625
NumArticles              0.000000
dtype: float64

For most machine learning projects, it’s reasonable to drop columns with more 
than 50% missing values, especially if there are plenty of other features.
High missingness usually means the feature will be hard to impute reliably and
won’t add robust predictive power.

Columns to drop:
	- Actor1Type2Code (97.5%)
	- Actor1Type3Code (99.9%)
	- Actor2Type2Code (98.3%)
	- Actor2Type3Code (99.9%)
	- Actor1Type1Code (55.2%)
	- Actor1CountryCode (46.1% — still too high for my taste, and difficult to impute)
	- Actor2CountryCode (56.6%)
	- Actor2Type1Code (67.2%)

One option for imputation would be to use the mode or rather the most frequent
value.
However, this is data from global events.
Imputation is always basically making up data and hoping it's a good guess.
Often, for numerical data, a mean or median is a good guess.
However, I am afraid in this case, it may not make much sense for some of the
columns.
For example, if the most frequent value is "USA", this will be filled in for
all rows where the value is missing.
But peprhaps there may be a good reason why the value is missing.
For example, if the value is missing, it may mean that the event is not
related to a country.
Because of this, I will treat the missing values as missing by introducing a
new category for unknown.
I will have to check how they encode this in general and which value can be
used for this.
Perhaps 0 is a good value for this in case it is not taken for anything else.


The column "SQLDATE" is a date in the format YYYYMMDD.
That integer is likely not very useful for modeling, so I will convert it to
more informative features such as year, month, day of year, day of week, and
whether it is a weekend or not.

I will also drop the intermediate date column.

In [97]:
# drop columns with roughly 50% or more missing values

# define columns to drop
columns_to_drop = [
    "Actor1Type2Code",
    "Actor1Type3Code",
    "Actor2Type2Code",
    "Actor2Type3Code",
    "Actor1Type1Code",
    "Actor1CountryCode",
    "Actor2CountryCode",
    "Actor2Type1Code"
]

# drop columns
train_df = train_df.drop(columns=columns_to_drop)

In [101]:
# check data again to get updated overview
print(train_df.info())
train_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 8000 entries, 9254 to 7270
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   SQLDATE                8000 non-null   Int64  
 1   MonthYear              8000 non-null   Int64  
 2   EventCode              8000 non-null   object 
 3   EventBaseCode          8000 non-null   object 
 4   EventRootCode          8000 non-null   object 
 5   QuadClass              8000 non-null   Int64  
 6   GoldsteinScale         8000 non-null   float64
 7   Actor1Code             7181 non-null   object 
 8   Actor1Name             7181 non-null   object 
 9   Actor2Code             5566 non-null   object 
 10  Actor2Name             5566 non-null   object 
 11  ActionGeo_CountryCode  7755 non-null   object 
 12  ActionGeo_ADM1Code     7755 non-null   object 
 13  ActionGeo_Lat          7741 non-null   float64
 14  ActionGeo_Long         7746 non-null   float64
 15  Action

,SQLDATE,MonthYear,EventCode,EventBaseCode,EventRootCode,QuadClass,GoldsteinScale,Actor1Code,Actor1Name,Actor2Code,Actor2Name,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,NumArticles
9254,20240328,202403,050,050,05,1,3.5,BUS,COMPANIES,None,None,SO,SO,6.0000,48.00000,SO,8
1561,20240515,202405,040,040,04,1,1.0,LEG,REPRESENTATIVES,FRAGOV,FRENCH,FR,FR00,48.8667,2.33333,-1456928,3
1670,20240405,202404,036,036,03,1,4.0,ESP,SPAIN,GOV,PRIME MINISTER,SP,SP,40.0000,-4.00000,SP,10
6087,20240209,202402,114,114,11,3,-2.0,USA,NORTH CAROLINA,LEG,CONGRESS,US,USNC,35.6411,-79.84310,NC,4
6669,20240815,202408,042,042,04,1,1.9,TZA,TANZANIA,KEN,NAIROBI,TZ,TZ,-6.0000,35.00000,TZ,1


Column in need for imputation and strategy:
- Actor1Code
    - Dtype: object
    - Example: "USA"
    - Strategy: Fill with "UNKNOWN"
- Actor1Name
    - Dtype: object
    - Example: "NORTH CAROLINA"
    - Strategy: Fill with "UNKNOWN"
- Actor2Code
    - Dtype: object
    - Example: "GOV"
    - Strategy: Fill with "UNKNOWN"
- Actor2Name
    - Dtype: object
    - Example: "PRIME MINISTER"
    - Strategy: Fill with "UNKNOWN"
- ActionGeo_CountryCode
    - Dtype: object
    - Example: "FR"
    - Strategy: Fill with "UNKNOWN"
- ActionGeo_ADM1Code
    - Dtype:    object
    - Example: "FR00"
    - Strategy: Fill with "UNKNOWN"
- ActionGeo_FeatureID
    - Dtype: object
    - Example: "TZ"
    - Strategy: Fill with "UNKNOWN"

There are two more columns in need of imputation and strategy:
ActionGeo_Lat and ActionGeo_Long, which are latitude and longitude of the event.

Two options:
- Impute with a value far outside the possible range (e.g., latitude 999,
longitude 999) or with a special flag value (e.g., -999, if the ML library
supports it).
This makes it clear to the model and downstream analysis that location is
missing—not just “somewhere ordinary.”
- Alternatively, it would be possible to use the mean or median
latitude/longitude, but this risks misleading the model to treat all
missing-location events as if they happened in a single place, which is
generally undesirable for geospatial modeling.

I will go with the first option, so here's the plan:
- ActionGeo_Lat
    - Dtype: float64
    - Example: 48.8667
    - Strategy: Impute with 999
- ActionGeo_Long
    - Dtype: float64
    - Example: 2.33333
    - Strategy: Impute with 999

In [102]:
# impute missing values

# define imputation strategy
imputation_strategy = {
    "Actor1Code": "UNKNOWN",
    "Actor1Name": "UNKNOWN",
    "Actor2Code": "UNKNOWN",
    "Actor2Name": "UNKNOWN",
    "ActionGeo_CountryCode": "UNKNOWN",
    "ActionGeo_ADM1Code": "UNKNOWN",
    "ActionGeo_FeatureID": "UNKNOWN",
    "ActionGeo_Lat": 999,
    "ActionGeo_Long": 999,
}

# fill missing values with new
train_df.fillna(imputation_strategy, inplace=True)

# check result
print(train_df.isna().mean())
train_df.head()

SQLDATE                  0.0
MonthYear                0.0
EventCode                0.0
EventBaseCode            0.0
EventRootCode            0.0
QuadClass                0.0
GoldsteinScale           0.0
Actor1Code               0.0
Actor1Name               0.0
Actor2Code               0.0
Actor2Name               0.0
ActionGeo_CountryCode    0.0
ActionGeo_ADM1Code       0.0
ActionGeo_Lat            0.0
ActionGeo_Long           0.0
ActionGeo_FeatureID      0.0
NumArticles              0.0
dtype: float64


,SQLDATE,MonthYear,EventCode,EventBaseCode,EventRootCode,QuadClass,GoldsteinScale,Actor1Code,Actor1Name,Actor2Code,Actor2Name,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,NumArticles
9254,20240328,202403,050,050,05,1,3.5,BUS,COMPANIES,UNKNOWN,UNKNOWN,SO,SO,6.0000,48.00000,SO,8
1561,20240515,202405,040,040,04,1,1.0,LEG,REPRESENTATIVES,FRAGOV,FRENCH,FR,FR00,48.8667,2.33333,-1456928,3
1670,20240405,202404,036,036,03,1,4.0,ESP,SPAIN,GOV,PRIME MINISTER,SP,SP,40.0000,-4.00000,SP,10
6087,20240209,202402,114,114,11,3,-2.0,USA,NORTH CAROLINA,LEG,CONGRESS,US,USNC,35.6411,-79.84310,NC,4
6669,20240815,202408,042,042,04,1,1.9,TZA,TANZANIA,KEN,NAIROBI,TZ,TZ,-6.0000,35.00000,TZ,1


Great! Now there are no missing values in the train_df.
I hope that this method makes any sense.
The only way to find out is to try it out.

## Take care of the columns for date

There is a column called "SQLDATE" which is a date in the format YYYYMMDD.
This is not very useful for modeling, so I will convert it to more informative
features such as year, month, day of year, day of week, and whether it is a
weekend or not.

I will also drop the intermediate date column.

Beyond this, there is a second column called "MonthYear" which seems to contain
redundant information.
It should probably be dropped.

In [ ]:
# Convert to datetime first
train_df['date'] = pd.to_datetime(train_df['SQLDATE'], format='%Y%m%d')

# Extract useful components
train_df['year'] = train_df['date'].dt.year
train_df['month'] = train_df['date'].dt.month
train_df['day_of_year'] = train_df['date'].dt.dayofyear  # 1-365
train_df['day_of_week'] = train_df['date'].dt.dayofweek  # 0=Monday, 6=Sunday
train_df['is_weekend'] = train_df['day_of_week'].isin([5, 6]).astype(int)

# Drop the intermediate date column if you want
train_df = train_df.drop('date', axis=1)

## Check if I need to One-Hot-Encode the categorical features

Right now, many of the categorical columns use integers to encode the values.
While this will probably work, it also introduces an order to the values.
A model may learn some patterns from this that don't really exist.

However, one-hot-encoding will increase the number of features by a lot.
This may be a problem if the number of features is too high.

Check which columns should not have an order and
if one-hot-encoding is feasible here by having a look at the number of
unique values in each column, then decide.

## See if any scaling or normalization is needed

## Divide into features and target

The target is the number of articles in the media `NumArticles`.

## Collect the steps and refactor them into a function

This exact same logic must be applied to the test_df, too, so the model will be
able to make meaningful predictions.

I will collect the steps and refactor them into a function.
This function can not only be used for the test_df, but it will also be useful
for later when I export this notebook as a script.

## Apply the same to test data